In [31]:
from time import *
from ephem import *
import math
import datetime
#
today=strftime("%Y/%m/%d")
horizon='00:00' # can also give in radians,

KAT=Observer()
KAT.long,KAT.lat='21:24:38.46','-30:43:17.34' # positive East,North
#KAT.date=today
#KAT.epoch=today
KAT.date='2017/11/19'
KAT.epoch=KAT.date
# print KAT.epoch
KAT.elev=1038            #-metres above MSL
KAT.temp=14             # temperature in C
KAT.pressure=0         # pressure in mbar set to zero to ignore refraction
KAT.horizon=horizon    # minimum elevation
dd=KAT.date.tuple()
print "Date %d/%d/%d UT %02d:%02d  LST %s Horizon %s"\
      %(dd[0],dd[1],dd[2],dd[3],dd[4],KAT.sidereal_time(),str(KAT.horizon))
print "Source\t\t  RA \t\t Dec \t  UTRise    UTTransit UTSet"
#
sun=Sun()
sun.compute(KAT)
#print "Sun  Alt",  sun.alt, "Az", sun.az
print "Sun \t\t%-12s %12s %-9s %-9s %-9s " \
      %(sun.ra,sun.dec,str(KAT.next_rising(sun))[-8:],\
        str(KAT.next_transit(sun))[-8:],str(KAT.next_setting(sun))[-8:])
#
moon=Moon()
moon.compute(KAT)
#print "Moon Alt", moon.alt,"Az", moon.az

print "Moon \t\t%-12s %12s %-9s %-9s %-9s " \
            % (moon.ra,moon.dec,str(KAT.next_rising(moon))[-8:],\
               str(KAT.next_transit(moon))[-8:],str(KAT.next_setting(moon))[-8:])




Date 2017/11/19 UT 00:00  LST 5:18:29.71 Horizon 0:00:00.0
Source		  RA 		 Dec 	  UTRise    UTTransit UTSet
Sun 		15:38:11.08   -19:26:43.5 03:29:50  10:19:47  17:10:06  
Moon 		16:06:47.02   -15:15:36.8 04:18:28  11:08:14  18:01:28  


In [30]:
def SAST(utstring): #YYYY/MM/DD hh:mm:ss format
    ut=datetime.datetime.strptime(utstring,"%Y/%m/%d %H:%M:%S")
    sast=ut+datetime.timedelta(hours=2)
    sasts= sast.strftime("%Y/%m/%d %H:%M:%S") # with seconds
    return sasts
    

In [31]:
def UT(day,time):# format YYYY/MM/DD and hh:mm - ignore seconds
    sast=datetime.datetime.strptime(day+time,"%Y/%m/%d%H:%M") # datetime format
    ut=sast- datetime.timedelta(hours=2)
    UTS=ut.strftime("%Y/%m/%d %H:%M:%S") # neat UT string with seconds
    return UTS


In [32]:
def LST(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    Set the time string to Hours:Minutes:Seconds
    """
    KAT.date=ut
    lst=str(KAT.sidereal_time())[:8]
    return lst


In [33]:
def JD(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    returns jd
    """
    jd=julian_date(ut)
    return jd

In [34]:
sunrise=KAT.next_rising(sun)
sunset=KAT.next_setting(sun)
print "Sunset ",localtime(sunset)
print "Sunrise",localtime(sunrise)
#

Sunset  2017-09-18 18:25:44
Sunrise 2017-09-18 06:31:35


In [35]:
local="13:45"+":00"
print local
datetime.time()


13:45:00


datetime.time(0, 0)

In [36]:
day="2016/05/14"   #string
time="00:15"       #string
sast=datetime.datetime.strptime(day+time,"%Y/%m/%d%H:%M") # datetime
print sast
ut=sast- datetime.timedelta(hours=2)
print ut
print "SAST",sast.strftime("%Y/%m/%d %H:%M:%S")," UT",ut.strftime("%Y/%m/%d %H:%M:%S")
                                


2016-05-14 00:15:00
2016-05-13 22:15:00
SAST 2016/05/14 00:15:00  UT 2016/05/13 22:15:00


In [37]:
print SAST("2016/05/13 22:46:00")

2016/05/14 00:46:00


In [38]:
x=UT("2016/05/16","13:00")
print x

2016/05/16 11:00:00


In [39]:
KAT.date="2016/06/18 17:45:00"   # UT string
str(KAT.sidereal_time())


'13:00:12.95'

In [40]:
date="2016/12/01"
time="20:00"
ut_test=UT(date,time)
print ut_test
lst=LST(ut_test)
print SAST(ut_test), ut_test, lst

2016/12/01 18:00:00
2016/12/01 20:00:00 2016/12/01 18:00:00 0:09:43.


In [41]:
inp=("dec40, J2000,7:00:00,40:00:00", "dec20, J2000, 7:00:00,20:00:00", "dec0, J2000, 7:00:00,00:00:00"\
     , "dec-20, J2000,7:00:00,-20:00:00", "dec-40, J2000,7:00:00,-40:00:00", "dec-60, J2000,7:00:00,-60:00:00")
for line in inp:
  if line[0] != "#":
    s=line.split(",") # split the line into strings
    source=s[0]
# next item is 'radec...'    
    ra=s[2]
    dec=s[3]
    dbline='"'+source+",f|J|Q,"+ra+","+dec+",,2000"+'"'
#    print dbline
    p=readdb(dbline)
#    p=FixedBody()
#    p._ra,p._dec=ra,dec
#    p._epoch='2000'
    p.compute(KAT)
    if p.neverup == True:
       print "%-11s  %12s %12s           Never Rises           " %(source,p.ra,p.dec)
    if p.circumpolar == True:
       transit=str(KAT.next_transit(p))[-8:]
       print "%-11s  %12s %12s          %9s  Circumpolar "  %(source,p.ra,p.dec,transit)
    if p.neverup == False and p.circumpolar == False:  # Source rises and sets
#      print source, p.ra, p.dec, p.alt, p.az,
      rises=str(KAT.next_rising(p))[-8:]
      transit=str(KAT.next_transit(p))[-8:]
      sets=str(KAT.next_setting(p))[-8:]
      uptime=((KAT.next_setting(p)-KAT.next_rising(p))%1)*24  # 
#calculation is in days, so make modulo 1 day, times 24 for hours.
      print "%-12s %12s %12s %-9s %-9s %-9s  %5.2f " % (source,p.ra,p.dec,rises,transit,sets,uptime)
#    for n in range(3,len(s)):    #comments at the end of the input line
#         print s[n],
#    print ""


dec40          7:01:11.05   39:58:18.4 20:50:30  00:50:20  04:50:10    7.99 
dec20          7:01:00.58   19:58:22.3 19:40:54  00:50:10  05:59:26   10.31 
dec0           7:00:52.69   -0:01:33.7 18:50:57  00:50:02  06:49:07   11.97 
dec-20         7:00:44.95  -20:01:30.0 18:00:59  00:49:54  07:38:49   13.63 
dec-40         7:00:35.05  -40:01:27.4 16:47:22  00:49:44  08:48:11   16.01 
dec-60         7:00:16.63  -60:01:25.2           00:49:26  Circumpolar 


In [42]:
print p.alt,p.az

60:41:52.1 180:00:00.0


In [43]:
KAT

<ephem.Observer date='2016/12/1 18:00:00' epoch='2017/9/18 00:00:00' lon='21:24:38.5' lat='-30:43:17.3' elevation=1038.0m horizon=0:00:00.0 temp=14.0C pressure=0.0mBar>

In [44]:
ut=today+" 13:47:20"
print LST(ut)
print JD(ut)

1:55:08.
2457815.07454


In [45]:
KAT.date=now()
print LST(KAT.date)

0:30:21.


In [46]:
str(localtime(now()))

'2017-03-02 14:22:46.000003'

In [47]:
now()

42795.015821759254

In [48]:
JD(ut)-now()-JD("1899/12/31 12:00:00")

0.058715277817100286

In [49]:
JD("2015/05/06 12:00:00")

2457149.0

In [50]:
print JD("1899/12/31 12:00:00"), LST("1899/12/31 12:00:00")

2415020.0 20:04:25


In [51]:
str(now()) # UT time now

'2017/3/2 12:22:47'

In [52]:
def tomorrow():
    tod=datetime.datetime.today().date() #today
    tom=tod+datetime.timedelta(1) # tomorrow
    toms=tom.strftime("%Y/%m/%d")
    return toms

In [53]:
print tomorrow()
tom=tomorrow()
print SAST(str(KAT.next_rising(sun,start=tomorrow())))

2017/03/03
2017/03/03 06:28:53


In [54]:
print "LST at sunset ", LST(str(sunset))
print "LST at sunrise", LST(str(KAT.next_rising(sun,start=tom)))
print "SAST Sunset   ", SAST(str(sunset))
print "SAST Sunrise  ", SAST(str(KAT.next_rising(sun,start=tom)))

LST at sunset  17:42:29
LST at sunrise 16:39:06
SAST Sunset    2017/09/18 18:25:44
SAST Sunrise   2017/03/03 06:28:53
